In [1]:
import numpy as np
import tensorflow as tf
from keras.models import Model
from keras.layers import Dropout,LSTM, Dense, Input, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from emo_utils import * 
import emoji

Using TensorFlow backend.


In [2]:
x_train, y_train = read_csv('train_emoji.csv')
x_test, y_test = read_csv('tesss.csv')

In [3]:
maxLen = len(max(x_train, key=len).split())

In [4]:
index=1
print(x_train[index], label_to_emoji(y_train[index]))

I am proud of your achievements 😄


In [5]:
y_oh_train = convert_to_one_hot(y_train,C=5)

In [6]:
def read_glove_vecs(glove_file):
    with open(glove_file, encoding="utf8" ) as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map
     
     


In [7]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B.50d.txt')

In [18]:
def sentence_to_indices(x, word_to_index, max_len):
    m = x.shape[0]
    x_indices = np.zeros((m,max_len))
    for i in range(m):
        sentence_words = [w.lower() for w in x[i].split()]
        j=0
        for w in sentence_words:
            x_indices[i,j] = word_to_index[w]
            j=j+1
    
    return x_indices

In [9]:
def pretrained_embedding_layer(word_to_vec, word_to_index):
    vocab_len = len(word_to_index)+1
    emb_dim = word_to_vec_map["boy"].shape[0]
    emb_matrix = np.zeros((vocab_len, emb_dim))
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]
    
    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)
    embedding_layer.build((None,))
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer
        

In [10]:
def Emojify(input_shape, word_to_vec_map, word_to_index):
    sentence_indices = Input(input_shape,dtype='int32')
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    embedding = embedding_layer(sentence_indices)
    X = LSTM(128, return_sequences=True)(embedding)
    X = Dropout(0.5)(X)
    X = LSTM(128, return_sequences=False)(embedding)
    X = Dropout(0.5)(X)
    X = Dense(5)(X)
    X = Activation('softmax')(X)
    model = Model(inputs = sentence_indices, outputs=X)
    return model
    
    

In [11]:
model = Emojify((maxLen,), word_to_vec_map, word_to_index)

model.summary()

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 10, 50)            20000050  
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               91648     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 645       
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
Total params: 20,092,343
Trainable params: 92,293
Non-trainable params: 20,000,050
___________________________________________________________

In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
x_train_indices = sentence_to_indices(x_train, word_to_index, maxLen)

In [15]:
y_train_oh = convert_to_one_hot(y_train, C=5)

In [16]:
model.fit(x_train_indices,y_train_oh, epochs=50, batch_size=32, shuffle=True)

Epoch 1/50
132/132 [==============================] - 4s 29ms/step - loss: 1.6013 - acc: 0.2273
Epoch 2/50
132/132 [==============================] - 0s 3ms/step - loss: 1.5622 - acc: 0.2727
Epoch 3/50
132/132 [==============================] - 0s 3ms/step - loss: 1.5173 - acc: 0.2879
Epoch 4/50
132/132 [==============================] - 0s 3ms/step - loss: 1.4889 - acc: 0.3409
Epoch 5/50
132/132 [==============================] - 0s 3ms/step - loss: 1.4482 - acc: 0.4015
Epoch 6/50
132/132 [==============================] - 0s 3ms/step - loss: 1.3907 - acc: 0.4015
Epoch 7/50
132/132 [==============================] - 0s 3ms/step - loss: 1.2858 - acc: 0.5682
Epoch 8/50
132/132 [==============================] - 0s 3ms/step - loss: 1.1996 - acc: 0.6212
Epoch 9/50
132/132 [==============================] - 0s 3ms/step - loss: 1.1503 - acc: 0.5227
Epoch 10/50
132/132 [==============================] - 0s 3ms/step - loss: 0.9939 - acc: 0.6742
Epoch 11/50
132/132 [===========================

In [20]:
x_test_indices = sentence_to_indices(x_test, word_to_index, max_len = maxLen)
y_test_oh = convert_to_one_hot(y_test, C = 5)
loss, acc = model.evaluate(x_test_indices, y_test_oh)
print()
print("Test accuracy = ", acc)

56/56 [==============================] - 0s 2ms/step

Test accuracy =  0.8571428571428571


In [25]:
x_test = np.array(['Lets have lunch together'])
X_test_indices = sentence_to_indices(x_test, word_to_index, maxLen)
print(x_test[0] +' '+  label_to_emoji(np.argmax(model.predict(X_test_indices))))

Lets have lunch together 🍴


In [1]:
import emoji

In [3]:
emoji.emojize(':smile:')

':smile:'